In [4]:
#Import Required Libraries
import pandas as pd
import numpy as np

#Import geocoder
from geopy import Nominatim
import geopy.distance

#Import Machine Learning Libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [5]:
df = pd.read_csv('../data/dataframe_recom_system_NR.csv')

In [6]:
df.head(1)

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Day,Month,Year,WeekNum,Rev,Return,B2B,AbsTrans,remove
0,0,537626,85232D,SET/3 DECOUPAGE STACKING TINS,3,2010-12-07 14:57:00,4.95,12347.0,Iceland,Tuesday,December,2010,49,14.85,0,1.0,14.85,False


In [421]:
df.CustomerID = df.CustomerID.astype(int)

In [422]:
df.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'Description', 'Quantity',
       'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'Day', 'Month',
       'Year', 'WeekNum', 'Rev', 'Return', 'B2B', 'AbsTrans', 'remove'],
      dtype='object')

In [423]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [424]:
df.describe()

,Quantity,UnitPrice,CustomerID,Year,WeekNum,Rev,Return,B2B,AbsTrans
count,306237.000000,306237.000000,306237.000000,306237.000000,306237.000000,306237.000000,306237.000000,306237.000000,306237.000000
mean,12.081476,2.737971,15310.257575,2010.936063,31.201269,19.336920,0.008291,0.993988,19.555612
std,46.786315,4.147309,1710.639896,0.244642,14.724763,93.480668,0.090677,0.077302,93.435163
min,-624.000000,0.000000,12347.000000,2010.000000,1.000000,-2432.700000,0.000000,0.000000,0.000000
25%,2.000000,0.950000,13952.000000,2011.000000,19.000000,4.200000,0.000000,1.000000,4.200000
50%,5.000000,1.650000,15235.000000,2011.000000,35.000000,10.400000,0.000000,1.000000,10.500000
75%,12.000000,3.250000,16813.000000,2011.000000,44.000000,17.850000,0.000000,1.000000,17.850000
max,12540.000000,649.500000,18287.000000,2011.000000,51.000000,38970.000000,1.000000,1.000000,38970.000000


In [425]:
df.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Day,Month,Year,WeekNum,Rev,Return,B2B,AbsTrans,remove
0,537626,85232D,SET/3 DECOUPAGE STACKING TINS,3,2010-12-07 14:57:00,4.95,12347,Iceland,Tuesday,December,2010,49,14.85,0,1.0,14.85,False


# Adding the Countries to Get Distance from UK

In [311]:
#Finding Customers with more than 1 Country listed
df_customer_stg = df.groupby('CustomerID')['Country'].unique()
custs2drop = df_customer_stg.loc[df_customer_stg.apply(lambda x:len(x)>1)]

df_custs2drop = pd.DataFrame(custs2drop)

df_custs2drop = df_custs2drop.reset_index()

custs2drop_list = df_custs2drop['CustomerID'].tolist()

df.shape

df[~df['CustomerID'].isin(custs2drop_list)].shape

df_clean = df[~df['CustomerID'].isin(custs2drop_list)]

In [333]:
cuntries = []
for i in df['Country'].unique():
    cuntries.append(i)

In [334]:
df_count = pd.DataFrame(cuntries)

In [335]:
df_count.head()

,0
0,Iceland
1,Finland
2,Italy
3,Norway
4,Bahrain


In [336]:
df_count.columns = ['Country']

df_count.head()

df_count['Latitude'] = df_count['Country'].apply(geolocator.geocode).apply(lambda x: x.latitude)
df_count['Longitude'] = df_count['Country'].apply(geolocator.geocode).apply(lambda x: x.longitude)

df_count.isna().sum()


df_count[df_count['Country'] == 'UNITED KINGDOM']


UK = (54.702354, -3.276575)


#Iterating between customers' countries to find the distance from UK
cust_country = range(df_count.shape[0])
distances = []
for s in cust_country:
    location = (df_count.iloc[s]['Latitude'],df_count.iloc[s]['Longitude'])
    distances.append(round(geopy.distance.vincenty(UK, location).km,2))

#Adding distances as feature
df_count['UKDist'] = distances   

df_count.sort_values(by='UKDist')


C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


,Country,Latitude,Longitude,UKDist
25,United Kingdom,54.702354,-3.276575,0.00
31,EIRE,52.865196,-7.979460,371.24
27,Netherlands,52.500170,5.748082,645.17
10,Belgium,50.640281,4.666715,701.44
12,Denmark,55.670249,10.333328,872.15
14,France,46.603354,1.888334,971.52
3,Norway,60.500021,9.099972,979.29
15,Germany,51.083420,10.423447,1004.01
7,Switzerland,46.798562,8.231974,1194.00
18,Sweden,59.674971,14.520858,1205.04


In [337]:
df_count.head(1)

,Country,Latitude,Longitude,UKDist
0,Iceland,64.984182,-18.105901,1407.87


# Obtaining to Features to Cluster By

In [426]:
df_summary = df.groupby(['CustomerID','Country']).agg({'Rev': ['sum','mean','min','max'],'InvoiceNo':'count','Quantity':'sum'})

In [427]:
df_summary.head()

Rev                           InvoiceNo Quantity
                        sum       mean    min     max     count      sum
CustomerID Country                                                      
12347      Iceland  4310.00  23.681319   5.04  249.60       182     2458
12348      Finland  1437.24  53.231111  13.20  150.00        27     2332
12349      Italy    1457.55  20.243750   6.64   63.80        72      630
12350      Norway    294.40  18.400000   8.50   25.20        16      196
12352      Norway   1265.41  18.077286   9.90   38.25        70      463

In [428]:
#I've not reset the index so that CustomerID and Country dont get mixed up in clustering
df_summary = df_summary.reset_index()
df_summary.head()

CustomerID  Country      Rev                           InvoiceNo Quantity
                           sum       mean    min     max     count      sum
0      12347  Iceland  4310.00  23.681319   5.04  249.60       182     2458
1      12348  Finland  1437.24  53.231111  13.20  150.00        27     2332
2      12349    Italy  1457.55  20.243750   6.64   63.80        72      630
3      12350   Norway   294.40  18.400000   8.50   25.20        16      196
4      12352   Norway  1265.41  18.077286   9.90   38.25        70      463

In [429]:
df_summary.columns = ['CustomerID','Country','Rev_Total','TransAvgVal','Trans_Min','Trans_Max','Trans_Num','TotQty']

In [430]:
df_summary.head()

,CustomerID,Country,Rev_Total,TransAvgVal,Trans_Min,Trans_Max,Trans_Num,TotQty
0,12347,Iceland,4310.00,23.681319,5.04,249.60,182,2458
1,12348,Finland,1437.24,53.231111,13.20,150.00,27,2332
2,12349,Italy,1457.55,20.243750,6.64,63.80,72,630
3,12350,Norway,294.40,18.400000,8.50,25.20,16,196
4,12352,Norway,1265.41,18.077286,9.90,38.25,70,463


In [431]:
df_final_stg = df_summary.merge(df_count, on='Country', how='left')

In [432]:
df_final_stg.head(10)

,CustomerID,Country,Rev_Total,TransAvgVal,Trans_Min,Trans_Max,Trans_Num,TotQty,Latitude,Longitude,UKDist
0,12347,Iceland,4310.00,23.681319,5.04,249.60,182,2458,64.984182,-18.105901,1407.87
1,12348,Finland,1437.24,53.231111,13.20,150.00,27,2332,63.246778,25.920916,1906.09
2,12349,Italy,1457.55,20.243750,6.64,63.80,72,630,42.638426,12.674297,1774.82
3,12350,Norway,294.40,18.400000,8.50,25.20,16,196,60.500021,9.099972,979.29
4,12352,Norway,1265.41,18.077286,9.90,38.25,70,463,60.500021,9.099972,979.29
5,12353,Bahrain,89.00,22.250000,11.60,39.80,4,20,26.155125,50.534461,5370.22
6,12354,Spain,1079.40,18.610345,8.50,54.08,58,530,39.326234,-4.838065,1713.38
7,12355,Bahrain,459.40,35.338462,17.70,120.00,13,240,26.155125,50.534461,5370.22
8,12356,Portugal,2487.43,42.886724,3.75,175.20,58,1573,40.033263,-7.889626,1666.76
9,12357,Switzerland,6207.67,47.386794,10.08,237.60,131,2708,46.798562,8.231974,1194.00


In [433]:
df_final = df_final_stg[['CustomerID','Rev_Total','TransAvgVal','Trans_Min','Trans_Max','Trans_Num','TotQty','UKDist']]

In [434]:
df_final.head(10)

,CustomerID,Rev_Total,TransAvgVal,Trans_Min,Trans_Max,Trans_Num,TotQty,UKDist
0,12347,4310.00,23.681319,5.04,249.60,182,2458,1407.87
1,12348,1437.24,53.231111,13.20,150.00,27,2332,1906.09
2,12349,1457.55,20.243750,6.64,63.80,72,630,1774.82
3,12350,294.40,18.400000,8.50,25.20,16,196,979.29
4,12352,1265.41,18.077286,9.90,38.25,70,463,979.29
5,12353,89.00,22.250000,11.60,39.80,4,20,5370.22
6,12354,1079.40,18.610345,8.50,54.08,58,530,1713.38
7,12355,459.40,35.338462,17.70,120.00,13,240,5370.22
8,12356,2487.43,42.886724,3.75,175.20,58,1573,1666.76
9,12357,6207.67,47.386794,10.08,237.60,131,2708,1194.00


In [437]:
df_final.set_index('CustomerID', inplace=True)

In [438]:
df_final.head(1)

,Rev_Total,TransAvgVal,Trans_Min,Trans_Max,Trans_Num,TotQty,UKDist
CustomerID,,,,,,,
12347,4310.0,23.681319,5.04,249.6,182,2458,1407.87


In [449]:
# Given the values are so large and hard to determine their meaning they will be inverted and normalised. 
# ie. the closer to 1, the more similar they are

df_final = pd.DataFrame(1/(1 + df_final))
df_final.head()

,Rev_Total,TransAvgVal,Trans_Min,Trans_Max,Trans_Num,TotQty,UKDist,labels,labelsPCA
CustomerID,,,,,,,,,
12347,0.000232,0.040516,0.165563,0.003990,0.005464,0.000407,0.000710,0.2,0.500000
12348,0.000695,0.018440,0.070423,0.006623,0.035714,0.000429,0.000524,0.5,0.333333
12349,0.000686,0.047073,0.130890,0.015432,0.013699,0.001585,0.000563,0.5,0.333333
12350,0.003385,0.051546,0.105263,0.038168,0.058824,0.005076,0.001020,0.5,0.333333
12352,0.000790,0.052418,0.091743,0.025478,0.014085,0.002155,0.001020,0.5,0.333333



## Model Build: Part 1 - K-means
The dataframe is now ready to build the model, containing all the required features. Initially, I will start with 6 clusters - a low, middle and high category for each B2B and B2C customers.

In [450]:
kmeans = KMeans(n_clusters=6)
customer_clusters = kmeans.fit(df_final)
customer_clusters.cluster_centers_

array([[ 3.86039426e-03,  4.98483281e-02,  1.26539149e-02,
         2.12851248e-02,  7.47551511e-02,  9.59929313e-03,
         1.00000000e+00,  4.73434418e-01,  3.44912299e-01],
       [ 2.56590416e-03,  5.14376154e-02,  8.75464178e-02,
         2.12895097e-02,  4.89215866e-02,  5.29784400e-03,
         8.33935692e-04,  4.58256620e-01,  3.47988408e-01],
       [ 2.28501196e-03,  1.03742526e-01,  7.97014260e+00,
         2.72603093e-02,  2.34926380e-02,  4.22858489e-03,
         1.00000000e+00,  4.62500000e-01,  3.54166667e-01],
       [ 3.48481647e-04,  1.41669286e-02, -1.66666667e+01,
         4.89949133e-03,  2.23502304e-02,  6.59754525e-04,
         1.00000000e+00,  3.50000000e-01,  4.16666667e-01],
       [ 4.27277715e-03,  1.26144585e-01,  6.18972227e-01,
         3.21462094e-02,  2.37392583e-02,  6.23594669e-03,
         1.00000000e+00,  4.85155138e-01,  3.40762881e-01],
       [ 3.18308363e-03,  5.97035475e-02, -3.35607356e+00,
         1.98897316e-02,  4.26148303e-02,  4.697760

## Model Build: Part 2 - PCA
With 7 features - it's worth comparing k-means with PCA.

In [451]:
# Principal Component 
pca = PCA(n_components=6)

principalComponents = pca.fit_transform(df_final)
principalDf = pd.DataFrame(data = principalComponents
             ,columns = ['pc1', 'pc2', 'pc3', 'pc4','pc5', 'pc6'])
principalDf.head()

,pc1,pc2,pc3,pc4,pc5,pc6
0,-0.045447,0.907920,-0.250856,0.164664,0.018216,-0.022791
1,-0.138534,0.898224,0.026969,-0.030630,-0.043134,-0.006419
2,-0.077161,0.899859,0.012235,-0.052479,-0.019452,-0.005650
3,-0.103037,0.897832,0.042667,-0.022143,0.001280,-0.005043
4,-0.116093,0.898009,0.012695,-0.054527,-0.010323,-0.005377


In [452]:
kmeans = KMeans(n_clusters=6)
october_clusters_PCA = kmeans.fit(principalDf)
october_clusters_PCA.cluster_centers_

array([[-1.41461582e-01, -1.01948279e-01,  7.81648944e-03,
         1.71086149e-02, -1.53238591e-02,  2.21554760e-05],
       [-1.68304601e+01, -6.34857818e-01, -1.73479987e-01,
        -2.36485133e-01,  2.74999413e-01, -8.25240804e-03],
       [ 7.78773815e+00,  1.53167777e-01, -1.96100329e-02,
         1.24612276e-01, -1.23742077e-01, -3.95450378e-03],
       [-1.17186703e-01,  8.98646646e-01,  2.89086157e-03,
        -9.30355026e-04, -1.67779136e-04, -2.53471447e-04],
       [ 4.47713430e-01, -8.40910725e-02, -1.78815782e-02,
        -3.68316405e-02,  3.45086441e-02,  7.56852381e-05],
       [-1.98006561e+00, -9.44832876e-02, -1.57090784e-02,
        -5.53868001e-02,  2.68233327e-02,  1.22231293e-04]])

## Model Addition to DataFrame

In [453]:
df_final['labels'] = customer_clusters.fit_predict(df_final)
df_final.head()

,Rev_Total,TransAvgVal,Trans_Min,Trans_Max,Trans_Num,TotQty,UKDist,labels,labelsPCA
CustomerID,,,,,,,,,
12347,0.000232,0.040516,0.165563,0.003990,0.005464,0.000407,0.000710,2,0.500000
12348,0.000695,0.018440,0.070423,0.006623,0.035714,0.000429,0.000524,2,0.333333
12349,0.000686,0.047073,0.130890,0.015432,0.013699,0.001585,0.000563,2,0.333333
12350,0.003385,0.051546,0.105263,0.038168,0.058824,0.005076,0.001020,2,0.333333
12352,0.000790,0.052418,0.091743,0.025478,0.014085,0.002155,0.001020,2,0.333333


In [454]:
df_final['labelsPCA'] = customer_clusters.fit_predict(principalDf)
df_final.head()

,Rev_Total,TransAvgVal,Trans_Min,Trans_Max,Trans_Num,TotQty,UKDist,labels,labelsPCA
CustomerID,,,,,,,,,
12347,0.000232,0.040516,0.165563,0.003990,0.005464,0.000407,0.000710,2,3
12348,0.000695,0.018440,0.070423,0.006623,0.035714,0.000429,0.000524,2,3
12349,0.000686,0.047073,0.130890,0.015432,0.013699,0.001585,0.000563,2,3
12350,0.003385,0.051546,0.105263,0.038168,0.058824,0.005076,0.001020,2,3
12352,0.000790,0.052418,0.091743,0.025478,0.014085,0.002155,0.001020,2,3


In [455]:
df_final[['labels','labelsPCA']] = df_final[['labels','labelsPCA']].replace([0, 1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6])

In [456]:
df_final.labels.value_counts()

4    2675
6    1156
3     418
1      60
5       8
2       2
Name: labels, dtype: int64

In [457]:
df_final.labelsPCA.value_counts()

2    2675
1    1156
4     418
6      60
5       8
3       2
Name: labelsPCA, dtype: int64

In [458]:
#Let's check if there is any relationship between the labels created by the two models:
df_final.groupby(["labels","labelsPCA"]).count()

,,Rev_Total,TransAvgVal,Trans_Min,Trans_Max,Trans_Num,TotQty,UKDist
labels,labelsPCA,,,,,,,
1,6,60,60,60,60,60,60,60
2,3,2,2,2,2,2,2,2
3,4,418,418,418,418,418,418,418
4,2,2675,2675,2675,2675,2675,2675,2675
5,5,8,8,8,8,8,8,8
6,1,1156,1156,1156,1156,1156,1156,1156
